# REST API Examples

In this notebook we explore searching for metadata from the REST API. The REST API provides a method to programmatically extract a JSON representation of the meta data from the API.

First we load some python dependancies that we will use as part of this notebook and set the variable `API_URL` to the location of the REST API.

In [1]:
import pandas as pd
import requests
API_URL = "https://mastapp.site/json"

Below we print the location of the endpoint we will query in this notebook:

In [2]:
print(f"REST API Endpoint: {API_URL}")

REST API Endpoint: https://mastapp.site/json


## Querying Shots with the REST API

We're going to use the python `requests` library to query metadata from the database. All we need to do to get a result is to query the database with a HTTP GET at the appropriate endpoint. For example, to get information about different experimental shots we can query the `/json/shots/` endpoint.

In [3]:
response = requests.get(f'{API_URL}/shots')
result = response.json()
print(f"Query returned status code: {response.status_code}")

Query returned status code: 200


The shots endpoint returns a JSON payload with a list of shots. Let's look at the first element from the payload:

In [4]:
print(result['items'][0])

{'title': 'Shot Dataset', 'shot_id': 11695, 'uuid': '219e5d59-b9dc-584f-b83b-018c47e8739d', 'url': 's3://mast/level1/shots/11695.zarr', 'endpoint_url': 'https://s3.echo.stfc.ac.uk', 'timestamp': '2004-12-13T11:54:00', 'preshot_description': '\n0.1T TF SHOT\n', 'postshot_description': '\nOK\n', 'campaign': 'M5', 'reference_shot': None, 'scenario': None, 'heating': None, 'pellets': False, 'rmp_coil': None, 'mcs_gdc_pre_shot': None, 'mcs_select_gas_group_pv1': None, 'mcs_select_gas_group_pv2': None, 'mcs_select_gas_group_pv3': None, 'mcs_select_gas_group_pv4': None, 'mcs_select_gas_group_pv5': None, 'mcs_select_gas_group_pv6': None, 'mcs_select_gas_group_pv7': None, 'mcs_select_gas_group_pv8': None, 'mcs_select_gas_group_pv9': None, 'mcs_select_lvps_tf_power_supply': None, 'mcs_select_cp3s_start_bank': None, 'mcs_select_fa1': None, 'mcs_select_fa2': None, 'mcs_select_mfps': None, 'mcs_select_efps': None, 'mcs_select_sfps': None, 'mcs_select_scs4': None, 'mcs_select_fa3': None, 'mcs_select

Each item in the list is a json object. This contains the meta-data items that corresponded to our query. In this case, each item contains information about a different MAST shot. Each item has lots of information about different shots, for example the shot ID, the campaign the shot was part of, the pre- and post-shot description by investigators.

For more information on the what's returned by the API you can look at the endpoint documentation:

https://mastapp.site/redoc


Of course, we can read all this JSON data directly into common python data analysis packages, for example, we can create a `pandas` dataframe directly from the endpoint data

In [5]:
df = pd.read_json(f'{API_URL}/shots', lines=True)['items'][0]
df = pd.DataFrame(df)
df.head()



,title,shot_id,uuid,url,endpoint_url,timestamp,preshot_description,postshot_description,campaign,reference_shot,...,generic_plasma_zeff_max_current,generic_plasma_zeff_max,generic_plasma_zeff_ruby_time,rad_magnetic_height_efit,current_range,divertor_config,plasma_shape,commissioner,facility,radii_magnetic:radius_efit
0,Shot Dataset,11695,219e5d59-b9dc-584f-b83b-018c47e8739d,s3://mast/level1/shots/11695.zarr,https://s3.echo.stfc.ac.uk,2004-12-13T11:54:00,\n0.1T TF SHOT\n,\nOK\n,M5,None,...,None,None,None,None,None,Conventional,None,None,MAST,None
1,Shot Dataset,11696,9ba69c96-34ac-5cd4-a1b8-8e9d7a44f871,s3://mast/level1/shots/11696.zarr,https://s3.echo.stfc.ac.uk,2004-12-13T12:07:00,\nSTANDARD 0.3T TF SHOT\n,\nOK\n,M5,None,...,None,None,None,None,None,Conventional,None,None,MAST,None
2,Shot Dataset,11697,6ecd5c59-6d16-5a0f-a715-be31a88ca34e,s3://mast/level1/shots/11697.zarr,https://s3.echo.stfc.ac.uk,2004-12-13T12:19:00,\nRAISE TO 0.5T\n,"\nOK, ALARMS ARE LOWER\n",M5,None,...,None,None,None,None,None,Conventional,None,None,MAST,None
3,Shot Dataset,11698,94366fda-b75e-5046-bb44-401844c563d8,s3://mast/level1/shots/11698.zarr,https://s3.echo.stfc.ac.uk,2004-12-13T12:31:00,\nRAISE TO .56T\n,\nSTILL ALARMS BUT LOWER AGAIN\n,M5,None,...,None,None,None,None,None,Conventional,None,None,MAST,None
4,Shot Dataset,11699,0e0f522d-a17f-5575-9bd0-d931231e8a71,s3://mast/level1/shots/11699.zarr,https://s3.echo.stfc.ac.uk,2004-12-13T12:45:00,\nRAISE TO .58T\n,\nOK\n,M5,None,...,None,None,None,None,None,Conventional,None,None,MAST,None


### Searching & Filtering Data

All REST API endpoints can take query parameters to filter the data returned. For example, we can return all shots for the `M9` campaign by using the approrpiate query string.

We can query for everything from the `M9` campaign by adding `?filters=campaign$eq:M9` to our query string.

In [6]:
df = pd.read_json(f'{API_URL}/shots?filters=campaign$eq:M9', lines=True)['items'][0]
df = pd.DataFrame(df)
df.head()

,title,shot_id,uuid,url,endpoint_url,timestamp,preshot_description,postshot_description,campaign,reference_shot,...,generic_plasma_zeff_max_current,generic_plasma_zeff_max,generic_plasma_zeff_ruby_time,rad_magnetic_height_efit,current_range,divertor_config,plasma_shape,commissioner,facility,radii_magnetic:radius_efit
0,Shot Dataset,28390,4310e5b2-f025-54c3-8cf2-cd84e2111f8f,s3://mast/level1/shots/28390.zarr,https://s3.echo.stfc.ac.uk,2012-03-06T14:47:00,"\nBC5, 300 ms, 3 V. D2 plenum 1536 mbar. For r...",\nOk.\n,M9,NaN,...,None,None,None,None,None,Conventional,Connected Double Null,None,MAST,None
1,Shot Dataset,28391,b6006fb2-cba1-5351-be52-11173f39db91,s3://mast/level1/shots/28391.zarr,https://s3.echo.stfc.ac.uk,2012-03-06T14:52:00,"\nBC5, 300 ms, 5 V. D2 plenum 1536 mbar. For r...",\nOk.\n,M9,NaN,...,None,None,None,None,None,Conventional,Connected Double Null,None,MAST,None
2,Shot Dataset,28392,63aefbf6-78c4-5684-b415-77820a06a1fe,s3://mast/level1/shots/28392.zarr,https://s3.echo.stfc.ac.uk,2012-03-06T15:03:00,"\nHL11, 300 ms, 2 V. He plenum 1047.\n",\nOk.\n,M9,NaN,...,None,None,None,None,None,Conventional,Connected Double Null,None,MAST,None
3,Shot Dataset,28393,6922f5c8-8b8f-5771-8b15-5a48c4b0ef19,s3://mast/level1/shots/28393.zarr,https://s3.echo.stfc.ac.uk,2012-03-06T15:09:00,"\nHL11, 300 ms, 3 V. He plenum 1047.\n",\nOk.\n,M9,NaN,...,None,None,None,None,None,Conventional,Connected Double Null,None,MAST,None
4,Shot Dataset,28394,17b22eb0-ea8b-5a4a-8061-c62681149034,s3://mast/level1/shots/28394.zarr,https://s3.echo.stfc.ac.uk,2012-03-06T15:13:00,"\nHL11, 300 ms, 5 V. He plenum 1047.\n",\nOk.\n,M9,NaN,...,None,None,None,None,None,Conventional,Connected Double Null,None,MAST,None


### Pagination 
The REST API responses are _paginated_, meaning that only a subset of the full items are returned with each query. Pagination is used to limit the total number of requests made by each user to prevent any single user overloading the server with huge data requests.

Pagination information is included in the response and corresponds to [RFC 8288](https://datatracker.ietf.org/doc/html/rfc8288). The response contains cursors at the bottom of the response:

 - `current_page` - this is the cursor relating to the current page of results.
 - `next_page` - this is the cursor for the next page of results.
 - `previous_page` - this is the cursor for the previous page of results, will result in `null` if on the first page.

You can control the pagination using the following options as query arguments:
 - `cursor=xx` to set the cursor for the displayed set of results
 - `size=xx` to set the number of results returned by each page.

In [7]:
response = requests.get(f'{API_URL}/shots?size=2&cursor=Pmk6MzAxMTE%3D')
result = response.json()
headers = response.headers

print("Current page cursor", result['current_page'])
print("Next page cursor", result['next_page'])
print("Previous page cursor", result['previous_page'])

df = pd.DataFrame(result['items'])
df

Current page cursor Pmk6MzAxMTE%3D
Next page cursor Pmk6MzAxMTM%3D
Previous page cursor PGk6MzAxMTI%3D


,title,shot_id,uuid,url,endpoint_url,timestamp,preshot_description,postshot_description,campaign,reference_shot,...,generic_plasma_zeff_max_current,generic_plasma_zeff_max,generic_plasma_zeff_ruby_time,rad_magnetic_height_efit,current_range,divertor_config,plasma_shape,commissioner,facility,radii_magnetic:radius_efit
0,Shot Dataset,30112,eefe2467-69b0-5fe7-9863-e21289daa135,s3://mast/level1/shots/30112.zarr,https://s3.echo.stfc.ac.uk,2013-09-09T11:42:00,\nReload 30110 - reduce IP flat top to 750 KA ...,\nPIC happy with data.\n,M9,30110,...,None,None,None,0.047497,700 kA,Conventional,Connected Double Null,None,MAST,0.922218
1,Shot Dataset,30113,742c78fa-3811-5404-93b3-5ab419d330bd,s3://mast/level1/shots/30113.zarr,https://s3.echo.stfc.ac.uk,2013-09-09T12:00:00,\nRestore 30111\n,\nShot OK for programme. Two good beams. Big l...,M9,30111,...,None,None,None,0.042220,700 kA,Conventional,Connected Double Null,None,MAST,0.907091
